### Ultralytics model Evaluator
Here I would calc the metrics and function for evaluating the Ultralytics models.

In [1]:
image_path = "/home/etaylor/images/processed_images/cannabis_patches/week9_15_06_2023/3x_regular/IMG_2157/IMG_2157_p4.png"
annotation_path = '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/annotations/yolo_split/labels/IMG_2157_p4.txt'
img_size = 512

In [2]:
from ultralytics import YOLO

yolov8_checkpoint = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/old/train16/weights/best.pt"
predictor = YOLO(yolov8_checkpoint)
outputs = predictor(image_path, imgsz=512, verbose=False, conf=0.3)

In [3]:
from src.segmentation.evaluation.ultralytics_evaluator import UltralyticsEvaluator
evaluator = UltralyticsEvaluator(num_classes=3, image_size=512)

In [4]:
gt_boxes = evaluator.parse_annotations(file_path=annotation_path)
pred_boxes = evaluator.parse_model_outputs(outputs)

In [5]:
class_distribution = {}

for box in gt_boxes:
    class_id = box['class_id']
    if class_id in class_distribution:
        class_distribution[class_id] += 1
    else:
        class_distribution[class_id] = 1
        
class_distribution

{1: 17, 2: 4, 0: 6}

In [6]:
tp = 3+10+3
fp = 2+2
fn = 2+4+1

precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"Precision: {precision}, Recall: {recall}")

Precision: 0.8, Recall: 0.6956521739130435


In [7]:
class0_precison = 3 / (3 + 5)
class0_recall = 3 / (3 + 3)
class1_precision = 10 / (10 + 3)
class1_recall = 10 / (10 + 7)
class2_precision = 3 / (3 + 0)
class2_recall = 3 / (3 + 1)

print(f"Class 0 Precision: {class0_precison}, Recall: {class0_recall}")
print(f"Class 1 Precision: {class1_precision}, Recall: {class1_recall}")
print(f"Class 2 Precision: {class2_precision}, Recall: {class2_recall}")

Class 0 Precision: 0.375, Recall: 0.5
Class 1 Precision: 0.7692307692307693, Recall: 0.5882352941176471
Class 2 Precision: 1.0, Recall: 0.75


In [8]:
eval_results = evaluator.evaluate_patch(gt_boxes, pred_boxes, single_class=False, iou_thresh=0.5)
eval_results

{'metrics': {'precision': 0.8,
  'recall': 0.6956521739130435,
  'class_wise_precision': array([      0.375,     0.76923,           1]),
  'class_wise_recall': array([        0.5,     0.58824,        0.75])},
 'confusion_matrix': array([[ 3,  1,  0,  2],
        [ 3, 10,  0,  4],
        [ 0,  0,  3,  1],
        [ 2,  2,  0,  0]]),
 'normalized_confusion_matrix': array([[        0.5,     0.16667,           0,     0.33333],
        [    0.17647,     0.58824,           0,     0.23529],
        [          0,           0,        0.75,        0.25],
        [        0.5,         0.5,           0,           0]], dtype=float32)}

In [9]:
import numpy as np
num_classes = 3

confusion_matrix = np.array([[ 3,  1,  0,  2],
                    [ 3, 10,  0,  4],
                    [ 0,  0,  3,  1],
                    [ 2,  2,  0,  0]])

precision = np.zeros(num_classes)
recall = np.zeros(num_classes)

In [10]:
for i in range(num_classes):
    tp = confusion_matrix[i, i]
    
    # Correctly compute fp by summing the misclassifications for the predicted class
    fp = np.sum(confusion_matrix[:, i]) - tp
    fn = np.sum(confusion_matrix[i, :]) - tp

    precision[i] = evaluator.calculate_precision(tp, fp)
    recall[i] = evaluator.calculate_recall(tp, fn)

# Compute overall precision and recall
overall_tp = np.trace(confusion_matrix[:num_classes, :num_classes])
overall_fp = np.sum(confusion_matrix[-1, :num_classes])
overall_fn = np.sum(confusion_matrix[:num_classes, -1])

overall_precision = evaluator.calculate_precision(overall_tp, overall_fp)
overall_recall = evaluator.calculate_recall(overall_tp, overall_fn)

In [11]:
print(f"Precision: {precision}, Recall: {recall}")
print(f"Overall Precision: {overall_precision}, Overall Recall: {overall_recall}")

Precision: [      0.375     0.76923           1], Recall: [        0.5     0.58824        0.75]
Overall Precision: 0.8, Overall Recall: 0.6956521739130435


In [ ]:
# Class 0 Precision: 0.375, Recall: 0.5
# Class 1 Precision: 0.7692307692307693, Recall: 0.5882352941176471
# Class 2 Precision: 1.0, Recall: 0.75

In [7]:
image_number = "IMG_2157"
images_dir = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/v0.3"
annotation_dir = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/annotations/yolo/labels/export_coco-instance_etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157_v0.3"

gt_boxes_image = evaluator.get_annotations_for_image_patches(image_number, annotation_dir)
gt_boxes_image

{'IMG_2157_p6.png': [{'bbox': [442.0, 217.000192, 470.0, 241.000192],
   'class_id': 1},
  {'bbox': [446.99991040000003,
    118.99996159999999,
    491.9998976,
    161.99997439999999],
   'class_id': 1},
  {'bbox': [364.0001024, 105.0001664, 393.00008959999997, 138.0001536],
   'class_id': 2},
  {'bbox': [397.00008959999997, 204.0002048, 424.0001024, 238.0001792],
   'class_id': 2},
  {'bbox': [385.0001408, 158.99991039999998, 411.00011520000004, 187.9998976],
   'class_id': 2},
  {'bbox': [335.000192, 5.999974399999999, 363.000192, 36.0], 'class_id': 2},
  {'bbox': [75.9998976, 449.0002176, 110.9999104, 480.0002304], 'class_id': 2},
  {'bbox': [88.000128, 102.0000256, 132.00012800000002, 145.0000384],
   'class_id': 1},
  {'bbox': [168.00002560000001, 28.0, 207.0000384, 69.9999744], 'class_id': 2},
  {'bbox': [477.9998464, 193.00016639999998, 510.9998336, 226.0001536],
   'class_id': 1}],
 'IMG_2157_p5.png': [{'bbox': [132.0002048,
    325.99982079999995,
    182.0001792,
    367.99

In [7]:
pred_boxes_image = evaluator.predict_and_parse_image_patches(image_number, images_dir, predictor)
pred_boxes_image

{'IMG_2157_p8.png': [{'bbox': [436.22903, 84.14053, 479.05765, 126.830444],
   'score': 0.8735967,
   'class_id': 2},
  {'bbox': [0.0, 75.44516, 46.420128, 129.79599],
   'score': 0.78576183,
   'class_id': 1},
  {'bbox': [311.60663, 56.520794, 371.4234, 106.26906],
   'score': 0.7736725,
   'class_id': 1},
  {'bbox': [322.47278, 102.6279, 368.9265, 139.65076],
   'score': 0.6919933,
   'class_id': 1},
  {'bbox': [184.3056, 216.57446, 241.22473, 269.8891],
   'score': 0.6757395,
   'class_id': 1},
  {'bbox': [15.015451, 0.18248558, 57.40055, 24.237692],
   'score': 0.5630582,
   'class_id': 0},
  {'bbox': [136.42522, 222.16681, 183.93361, 269.36145],
   'score': 0.56120545,
   'class_id': 0},
  {'bbox': [45.261173, 22.489883, 89.077515, 67.5351],
   'score': 0.5326675,
   'class_id': 1},
  {'bbox': [398.6551, 0.0, 458.67108, 9.762722],
   'score': 0.4587633,
   'class_id': 1},
  {'bbox': [227.96896, 0.0, 292.63907, 12.505738],
   'score': 0.45334408,
   'class_id': 1},
  {'bbox': [435.

In [8]:
image_results = evaluator.evaluate_image(gt_boxes_image, pred_boxes_image, single_class=False, iou_thresh=0.5)
image_results

Evaluating patch IMG_2157_p6.png
Patch IMG_2157_p6.png conf matrix:
 [[0 0 0 0]
 [2 1 0 1]
 [0 0 6 0]
 [0 1 0 0]]
Evaluating patch IMG_2157_p5.png
Patch IMG_2157_p5.png conf matrix:
 [[ 0  0  0  0]
 [ 2 13  0  3]
 [ 0  0  2  0]
 [ 3  5  0  0]]
Evaluating patch IMG_2157_p4.png
Patch IMG_2157_p4.png conf matrix:
 [[ 3  0  0  0]
 [ 3 12  0  1]
 [ 0  0  3  1]
 [ 2  3  0  0]]
Evaluating patch IMG_2157_p7.png
Patch IMG_2157_p7.png conf matrix:
 [[0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 2 0]]
Evaluating patch IMG_2157_p8.png
Patch IMG_2157_p8.png conf matrix:
 [[0 1 0 0]
 [0 4 0 0]
 [0 0 1 0]
 [4 2 0 0]]
Evaluating patch IMG_2157_p10.png
Patch IMG_2157_p10.png conf matrix:
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 4 1]
 [0 2 0 0]]
Evaluating patch IMG_2157_p0.png
Patch IMG_2157_p0.png conf matrix:
 [[0 0 0 0]
 [0 1 0 1]
 [0 0 2 1]
 [0 0 1 0]]
Evaluating patch IMG_2157_p1.png
Patch IMG_2157_p1.png conf matrix:
 [[0 0 0 0]
 [3 7 0 2]
 [0 0 2 0]
 [1 0 0 0]]
Evaluating patch IMG_2157_p3.png
Patch IMG_2157_p3.png

{'metrics': {'precision': 0.7272727272727273,
  'recall': 0.8,
  'class_wise_precision': array([       0.25,     0.76271,     0.88571]),
  'class_wise_recall': array([          1,      0.7377,     0.88571])},
 'confusion_matrix': array([[ 4,  2,  0,  0],
        [10, 45,  0, 16],
        [ 1,  0, 31,  4],
        [12, 14,  4,  0]]),
 'normalized_confusion_matrix': array([[    0.66667,     0.33333,           0,           0],
        [    0.14085,      0.6338,           0,     0.22535],
        [   0.027778,           0,     0.86111,     0.11111],
        [        0.4,     0.46667,     0.13333,           0]], dtype=float32)}

In [11]:
images_dir = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/v0.1"
annotation_dir = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/yolo_split/labels"


gt_boxes_dataset = evaluator.get_annotations_for_dataset(annotation_dir)
pred_boxes_dataset = evaluator.predict_and_parse_dataset(images_dir, predictor)

In [12]:
results_dataset = evaluator.evaluate_dataset(gt_boxes_dataset, pred_boxes_dataset, single_class=False, iou_thresh=0.5)
results_dataset

Evaluating image IMG_0019
Evaluating patch IMG_0019_p0.png
Patch IMG_0019_p0.png conf matrix:
 [[1 2 0 3]
 [0 2 0 3]
 [0 0 0 0]
 [0 0 0 0]]
Evaluating patch IMG_0019_p1.png
Patch IMG_0019_p1.png conf matrix:
 [[4 2 0 3]
 [1 7 0 2]
 [0 0 0 0]
 [2 3 0 0]]
Evaluating patch IMG_0019_p10.png
Patch IMG_0019_p10.png conf matrix:
 [[4 0 0 1]
 [1 1 0 0]
 [0 0 0 1]
 [2 0 0 0]]
Evaluating patch IMG_0019_p2.png
Patch IMG_0019_p2.png conf matrix:
 [[4 1 0 5]
 [3 3 0 2]
 [0 0 0 2]
 [4 3 0 0]]
Evaluating patch IMG_0019_p3.png
Patch IMG_0019_p3.png conf matrix:
 [[3 0 0 5]
 [2 4 0 3]
 [0 0 0 0]
 [5 0 0 0]]
Evaluating patch IMG_0019_p4.png
Patch IMG_0019_p4.png conf matrix:
 [[ 9  3  0 10]
 [ 4  7  0  4]
 [ 0  0  0  0]
 [ 3  0  0  0]]
Evaluating patch IMG_0019_p6.png
Patch IMG_0019_p6.png conf matrix:
 [[7 1 0 3]
 [2 6 0 4]
 [0 0 0 0]
 [7 2 0 0]]
Evaluating patch IMG_0019_p5.png
Patch IMG_0019_p5.png conf matrix:
 [[10  3  0  4]
 [ 7  9  0  4]
 [ 0  0  0  0]
 [ 2  2  0  0]]
Evaluating patch IMG_0019_p7

{'metrics': {'precision': 0.7033333333333334,
  'recall': 0.6698412698412698,
  'class_wise_precision': array([    0.65524,     0.74178,     0.72222]),
  'class_wise_recall': array([    0.70928,     0.70222,        0.26])},
 'confusion_matrix': array([[344,  62,   2, 141],
        [184, 474,   3, 201],
        [  3,  11,  26,  74],
        [181, 165,  10,   0]]),
 'normalized_confusion_matrix': array([[    0.62659,     0.11293,    0.003643,     0.25683],
        [    0.21346,     0.54988,   0.0034803,     0.23318],
        [   0.026316,    0.096491,     0.22807,     0.64912],
        [    0.50843,     0.46348,     0.02809,           0]], dtype=float32)}

In [13]:
# get the dist of ground truth
dist = {}

# Iterate over each image in the dataset
for image_patches in gt_boxes_dataset.values():
    # Iterate over each patch in the image
    for patch_boxes in image_patches.values():
        # Iterate over each bounding box in the patch
        for box in patch_boxes:
            class_id = box['class_id']
            if class_id in dist:
                dist[class_id] += 1
            else:
                dist[class_id] = 1
dist

{0: 549, 1: 862, 2: 114}

### Background Class in Confusion Matrix

1.  **False Positives (FP)**: These occur when the model predicts an object where there is none in the ground truth. In the confusion matrix, this is represented in the last row corresponding to the background (usually `-1` or `num_classes`).
    
2.  **False Negatives (FN)**: These occur when an actual object (ground truth) is not detected by the model. In the confusion matrix, this is represented in the last column corresponding to the background.

### Mock Examples

### Detectron2 Models Evaluator

In [1]:
# from src.annotation_handling import annotation_handler

# dataset_name_train = "etaylor/cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157"
# release_train = "v0.3"

# _, train_export_json_path, train_saved_images_path = annotation_handler.convert_segments_to_coco_format(
#         dataset_name=dataset_name_train, 
#         release_version=release_train, 
#     )

RuntimeError: KeyboardInterrupt: 

In [1]:
detectron2_model_config = {
        "model_name": "mask_rcnn_R_50_FPN_3x",
        "checkpoint": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/model_final.pth",
        "yaml_file": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml"
    } 

In [2]:
image_path = "/home/etaylor/images/processed_images/cannabis_patches/week9_15_06_2023/3x_regular/IMG_2157/IMG_2157_p4.png"
coco_annotations_file = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/annotations/export_coco-instance_etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157_v0.3.json"
img_size = 512

In [3]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

cfg = get_cfg()
# load config
cfg.merge_from_file(detectron2_model_config['yaml_file'])

# load checkpoint
cfg.MODEL.WEIGHTS = detectron2_model_config['checkpoint']
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# define predictor
predictor = DefaultPredictor(cfg)

Loading config /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [4]:
from src.segmentation.evaluation.detectron2_evaluator import Detectron2Evaluator

det_evaluator = Detectron2Evaluator(num_classes=3, coco_annotations_file_path=coco_annotations_file)

In [5]:
dt_gt_boxes = det_evaluator.parse_annotations(file_path="IMG_2157_p4.png")
dt_gt_boxes

[{'bbox': [325, 195, 370, 243], 'class_id': 1},
 {'bbox': [310, 370, 367, 420], 'class_id': 1},
 {'bbox': [471, 253, 512, 294], 'class_id': 1},
 {'bbox': [377, 125, 428, 176], 'class_id': 1},
 {'bbox': [248, 362, 299, 413], 'class_id': 1},
 {'bbox': [145, 81, 195, 132], 'class_id': 1},
 {'bbox': [225, 83, 257, 116], 'class_id': 2},
 {'bbox': [283, 251, 330, 301], 'class_id': 1},
 {'bbox': [166, 344, 205, 387], 'class_id': 1},
 {'bbox': [85, 83, 134, 131], 'class_id': 1},
 {'bbox': [475, 213, 511, 246], 'class_id': 1},
 {'bbox': [417, 27, 438, 47], 'class_id': 2},
 {'bbox': [174, 135, 218, 183], 'class_id': 2},
 {'bbox': [446, 170, 492, 213], 'class_id': 1},
 {'bbox': [167, 58, 196, 84], 'class_id': 2},
 {'bbox': [460, 84, 497, 114], 'class_id': 0},
 {'bbox': [190, 188, 238, 233], 'class_id': 0},
 {'bbox': [191, 257, 232, 295], 'class_id': 1},
 {'bbox': [186, 29, 232, 75], 'class_id': 1},
 {'bbox': [455, 374, 504, 424], 'class_id': 0},
 {'bbox': [247, 170, 295, 216], 'class_id': 1},
 {'

In [6]:
class_distribution = {}

for box in dt_gt_boxes:
    class_id = box['class_id']
    if class_id in class_distribution:
        class_distribution[class_id] += 1
    else:
        class_distribution[class_id] = 1
        
class_distribution

{1: 16, 2: 4, 0: 3}

In [7]:
import cv2
outputs = predictor(cv2.imread(image_path))

/home/etaylor/.conda/envs/detectron/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
dt_pred_boxes = det_evaluator.parse_model_outputs(outputs)

In [9]:
class_distribution = {}

for box in dt_pred_boxes:
    class_id = box['class_id']
    if class_id in class_distribution:
        class_distribution[class_id] += 1
    else:
        class_distribution[class_id] = 1
        
class_distribution

{1: 16, 2: 4, 0: 5}

In [10]:
matches, misclassifications, false_positives, false_negatives = det_evaluator.match_predictions(dt_gt_boxes, dt_pred_boxes, 0.5)

In [11]:
matches

[(3, 0, 0.9615659871121797),
 (6, 1, 0.9535614360462535),
 (0, 2, 0.9738277495492108),
 (1, 3, 0.9443915989144545),
 (8, 4, 0.9596564641418112),
 (5, 5, 0.9564180659069033),
 (4, 6, 0.9499731639585457),
 (9, 7, 0.9719430313629358),
 (11, 8, 0.9049533996000407),
 (2, 9, 0.9491275084227468),
 (7, 11, 0.9880752881816552),
 (17, 12, 0.9332203174401766),
 (16, 13, 0.9382712037719795),
 (10, 14, 0.9536168326968592),
 (14, 15, 0.8417485816852854),
 (18, 16, 0.7891871086365785),
 (13, 17, 0.9408637772289744),
 (15, 18, 0.9019954713563347),
 (12, 20, 0.9124581909474249)]

In [12]:
misclassifications

[(19, 10, 0.8895517335081942),
 (21, 19, 0.849404986749232),
 (20, 21, 0.7915501299256551),
 (22, 22, 0.7836707224900573)]

In [13]:
false_positives

[{'bbox': [106.9606, 229.36613, 144.90823, 267.99908],
  'score': 0.7756821,
  'class_id': 1}]

In [14]:
dt_eval_results = det_evaluator.evaluate_patch(dt_gt_boxes, dt_pred_boxes, iou_thresh=0.5, single_class=False)

In [15]:
dt_eval_results

{'metrics': {'precision': 0.95,
  'recall': 1.0,
  'class_wise_precision': array([1.        , 0.92857143, 1.        ]),
  'class_wise_recall': array([1., 1., 1.])},
 'confusion_matrix': array([[ 2,  1,  0,  0],
        [ 3, 13,  0,  0],
        [ 0,  0,  4,  0],
        [ 0,  1,  0,  0]]),
 'normalized_confusion_matrix': array([[0.6666667 , 0.33333334, 0.        , 0.        ],
        [0.1875    , 0.8125    , 0.        , 0.        ],
        [0.        , 0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        ]], dtype=float32)}

In [6]:
import re

patches_gt_boxes = {}
image_number = "IMG_2157"
pattern = re.compile(rf"^{image_number}_p\d+\.\w+$")  # Regex pattern to match specific file names

# Iterate through all images in the COCO dataset to find matches
for img in det_evaluator.coco_data['images']:
    if image_number in img['file_name']:  # Check if the base image name is in the file name
        print(img['file_name'])
        gt_boxes = det_evaluator.parse_annotations(img['file_name'])
        patches_gt_boxes[img['file_name']] = gt_boxes

IMG_2157.jpg
IMG_2157_p6.png
IMG_2157_p5.png
IMG_2157_p4.png
IMG_2157_p7.png
IMG_2157_p8.png
IMG_2157_p10.png
IMG_2157_p0.png
IMG_2157_p1.png
IMG_2157_p3.png
IMG_2157_p2.png
IMG_2157_p9.png


In [7]:
patches_gt_boxes = det_evaluator.get_annotations_for_image_patches(image_number="IMG_2157")
images_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/v0.1"
patches_pred_boxes = det_evaluator.predict_and_parse_image_patches("IMG_2157", images_path, predictor)

IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157
IMG_2157


/home/etaylor/.conda/envs/detectron/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
class_distribution = {}

for boxes in patches_gt_boxes.values():
    for box in boxes:
        class_id = box['class_id']
        if class_id in class_distribution:
            class_distribution[class_id] += 1
        else:
            class_distribution[class_id] = 1
        
class_distribution

{1: 71, 2: 36, 0: 6}

In [9]:
result = det_evaluator.evaluate_image(patches_gt_boxes_dict=patches_gt_boxes, patches_pred_boxes_dict=patches_pred_boxes, iou_thresh=0.5)
result

Evaluating patch IMG_2157_p6.png
Patch IMG_2157_p6.png conf matrix:
 [[0 0 0 0]
 [0 4 0 0]
 [0 0 6 0]
 [0 0 0 0]]
Evaluating patch IMG_2157_p5.png
Patch IMG_2157_p5.png conf matrix:
 [[ 0  0  0  0]
 [ 0 18  0  0]
 [ 0  0  2  0]
 [ 0  1  0  0]]
Evaluating patch IMG_2157_p4.png
Patch IMG_2157_p4.png conf matrix:
 [[ 2  1  0  0]
 [ 3 13  0  0]
 [ 0  0  4  0]
 [ 0  1  0  0]]
Evaluating patch IMG_2157_p7.png
Patch IMG_2157_p7.png conf matrix:
 [[0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 0]]
Evaluating patch IMG_2157_p8.png
Patch IMG_2157_p8.png conf matrix:
 [[1 0 0 0]
 [0 4 0 0]
 [0 0 1 0]
 [0 1 0 0]]
Evaluating patch IMG_2157_p10.png
Patch IMG_2157_p10.png conf matrix:
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 5 0]
 [0 0 0 0]]
Evaluating patch IMG_2157_p0.png
Patch IMG_2157_p0.png conf matrix:
 [[0 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 0]]
Evaluating patch IMG_2157_p1.png
Patch IMG_2157_p1.png conf matrix:
 [[ 0  0  0  0]
 [ 0 12  0  0]
 [ 0  0  2  0]
 [ 0  0  0  0]]
Evaluating patch IMG_2157_p3.png
Patch

{'metrics': {'precision': 0.9727272727272728,
  'recall': 0.9907407407407407,
  'class_wise_precision': array([1.        , 0.95714286, 1.        ]),
  'class_wise_recall': array([1.        , 0.98529412, 1.        ])},
 'confusion_matrix': array([[ 4,  2,  0,  0],
        [ 3, 67,  0,  1],
        [ 0,  0, 36,  0],
        [ 0,  3,  0,  0]]),
 'normalized_confusion_matrix': array([[0.6666667 , 0.33333334, 0.        , 0.        ],
        [0.04225352, 0.943662  , 0.        , 0.01408451],
        [0.        , 0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        ]], dtype=float32)}

In [10]:
dt_test_evaluator = Detectron2Evaluator(num_classes=3, coco_annotations_file_path="/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1.json")

In [ ]:
# get annotations for a single image
patches_gt_boxes = dt_test_evaluator.get_annotations_for_image_patches(image_number="IMG_2271")

In [ ]:
patches_gt_boxes['IMG_2271_p1.png']

[{'bbox': [86, 109, 111, 143], 'class_id': 1}]

In [8]:
images_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/v0.1"
patches_pred_boxes = det_evaluator.predict_and_parse_image_patches("IMG_2271", images_path, predictor)

/home/etaylor/.conda/envs/detectron/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [18]:
result = det_evaluator.evaluate_image(patches_gt_boxes_dict=patches_gt_boxes, patches_pred_boxes_dict=patches_pred_boxes, iou_thresh=0.5)

Evaluating patch IMG_2271_p0.png
Evaluating patch IMG_2271_p10.png
Evaluating patch IMG_2271_p11.png
Evaluating patch IMG_2271_p1.png
Evaluating patch IMG_2271_p2.png
Evaluating patch IMG_2271_p3.png
Evaluating patch IMG_2271_p4.png
Evaluating patch IMG_2271_p5.png
Evaluating patch IMG_2271_p6.png
Evaluating patch IMG_2271_p7.png
Evaluating patch IMG_2271_p8.png
Evaluating patch IMG_2271_p9.png


In [19]:
result

NameError: name 'result' is not defined

In [1]:
detectron2_model_config = {
        "model_name": "mask_rcnn_R_50_FPN_3x",
        "checkpoint": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/model_final.pth",
        "yaml_file": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml"
    }

In [11]:
image_path = "/home/etaylor/images/processed_images/cannabis_patches/week9_15_06_2023/3x_regular/IMG_2157/IMG_2157_p4.png"
coco_annotations_file = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1.json"
img_size = 512

In [3]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

cfg = get_cfg()
# load config
cfg.merge_from_file(detectron2_model_config['yaml_file'])

# load checkpoint
cfg.MODEL.WEIGHTS = detectron2_model_config['checkpoint']
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# define predictor
predictor = DefaultPredictor(cfg)

Loading config /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [12]:
from src.segmentation.evaluation.detectron2_evaluator import Detectron2Evaluator
det_evaluator = Detectron2Evaluator(num_classes=3, coco_annotations_file_path=coco_annotations_file)

dataset_images_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/v0.1"

In [13]:
dataset_gt_boxes = det_evaluator.get_annotations_for_dataset(dataset_images_path)

IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2198
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_2285
IMG_0542
IMG_0542
IMG_0542
IMG_0542
IMG_0542
IMG_0542
IMG_0542
IMG_0542
IMG_0542
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_1093
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_0019
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_2271
IMG_0058
IMG_0058
IMG_0058
IMG_0058
IMG_0058
IMG_0058
IMG_0058
IMG_0058
IMG_0058
IMG_1753
IMG_1753
IMG_1753
IMG_1753
IMG_1753
IMG_1753
IMG_1079
IMG_1079
IMG_1079
IMG_1079
IMG_1079
IMG_1079
IMG_1079
IMG_1079
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096
IMG_1096


In [14]:
dataset_pred_boxes = det_evaluator.predict_and_parse_dataset(dataset_images_path, predictor)

In [16]:
class_distribution = {}

# Iterate over each image in the dataset
for image_patches in dataset_gt_boxes.values():
    # Iterate over each patch in the image
    for patch_boxes in image_patches.values():
        # Iterate over each bounding box in the patch
        for box in patch_boxes:
            class_id = box['class_id']
            if class_id in class_distribution:
                class_distribution[class_id] += 1
            else:
                class_distribution[class_id] = 1
                
class_distribution

{1: 862, 0: 549, 2: 114}

In [17]:
result = det_evaluator.evaluate_dataset(dataset_gt_boxes_dict=dataset_gt_boxes, dataset_pred_boxes_dict=dataset_pred_boxes, iou_thresh=0.5)

Evaluating image IMG_2198
Evaluating patch IMG_2198_p0.png
Patch IMG_2198_p0.png conf matrix:
 [[0 0 0 0]
 [0 4 0 0]
 [0 0 0 0]
 [0 3 0 0]]
Evaluating patch IMG_2198_p10.png
Patch IMG_2198_p10.png conf matrix:
 [[2 2 0 0]
 [0 4 0 1]
 [0 0 0 0]
 [0 4 0 0]]
Evaluating patch IMG_2198_p11.png
Patch IMG_2198_p11.png conf matrix:
 [[2 0 0 0]
 [0 4 1 0]
 [0 0 2 1]
 [0 1 1 0]]
Evaluating patch IMG_2198_p12.png
Patch IMG_2198_p12.png conf matrix:
 [[0 2 0 1]
 [0 7 0 0]
 [0 0 1 0]
 [1 6 0 0]]
Evaluating patch IMG_2198_p1.png
Patch IMG_2198_p1.png conf matrix:
 [[1 1 0 0]
 [0 3 0 1]
 [0 0 0 0]
 [2 3 0 0]]
Evaluating patch IMG_2198_p2.png
Patch IMG_2198_p2.png conf matrix:
 [[ 0  5  0  0]
 [ 0 15  0  1]
 [ 0  0  0  0]
 [ 0  3  0  0]]
Evaluating patch IMG_2198_p3.png
Patch IMG_2198_p3.png conf matrix:
 [[ 1  6  0  0]
 [ 0 16  0  4]
 [ 0  0  0  0]
 [ 0  4  0  0]]
Evaluating patch IMG_2198_p4.png
Patch IMG_2198_p4.png conf matrix:
 [[ 0  3  0  1]
 [ 0 18  0  1]
 [ 0  0  0  0]
 [ 1  2  0  0]]
Evaluati

Patch IMG_0019_p2.png conf matrix:
 [[5 2 0 3]
 [1 6 0 1]
 [0 1 0 1]
 [1 4 0 0]]
Evaluating patch IMG_0019_p3.png
Patch IMG_0019_p3.png conf matrix:
 [[3 2 0 3]
 [0 7 0 2]
 [0 0 0 0]
 [1 5 0 0]]
Evaluating patch IMG_0019_p4.png
Patch IMG_0019_p4.png conf matrix:
 [[ 9  8  0  5]
 [ 0 10  0  5]
 [ 0  0  0  0]
 [ 0  2  0  0]]
Evaluating patch IMG_0019_p5.png
Patch IMG_0019_p5.png conf matrix:
 [[ 9  7  0  1]
 [ 1 16  0  3]
 [ 0  0  0  0]
 [ 1  3  0  0]]
Evaluating patch IMG_0019_p6.png
Patch IMG_0019_p6.png conf matrix:
 [[6 3 0 2]
 [3 7 0 2]
 [0 0 0 0]
 [0 2 0 0]]
Evaluating patch IMG_0019_p7.png
Patch IMG_0019_p7.png conf matrix:
 [[ 4  4  0 10]
 [ 0 10  0  0]
 [ 0  0  0  0]
 [ 2  2  0  0]]
Evaluating patch IMG_0019_p8.png
Patch IMG_0019_p8.png conf matrix:
 [[11  3  0  4]
 [ 5  3  0  0]
 [ 0  0  0  0]
 [ 0  5  0  0]]
Evaluating patch IMG_0019_p9.png
Patch IMG_0019_p9.png conf matrix:
 [[10  5  0  1]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 3  0  0  0]]
Evaluating image IMG_2271
Evaluating patc

In [18]:
result

{'metrics': {'precision': 0.77119341563786,
  'recall': 0.7484025559105432,
  'class_wise_precision': array([0.76470588, 0.77505828, 0.74509804]),
  'class_wise_recall': array([0.65546218, 0.83647799, 0.38      ])},
 'confusion_matrix': array([[234, 191,   1, 123],
        [ 60, 665,   7, 130],
        [  2,  12,  38,  62],
        [ 72, 193,  13,   0]]),
 'normalized_confusion_matrix': array([[0.4262295 , 0.34790528, 0.00182149, 0.22404371],
        [0.06960557, 0.7714617 , 0.00812065, 0.15081206],
        [0.01754386, 0.10526316, 0.33333334, 0.54385966],
        [0.2589928 , 0.6942446 , 0.04676259, 0.        ]], dtype=float32)}

In [20]:
# get dataset trichome dist
from src.annotation_handling.segmentsai_handler import SegmentsAIHandler
segments_handler = SegmentsAIHandler()
import config

dataset_identifier = "etaylor/cannabis_patches_test_26-04-2024_15-44-44"
# Fetch the samples
samples = segments_handler.client.get_samples(dataset_identifier)

# Initialize counters for each trichome type
distribution = {
    'clear': 0,
    'cloudy': 0,
    'amber': 0
}

# Fetch labels for each sample and count annotations
for sample in samples:
    label = segments_handler.client.get_label(sample.uuid)
    for annotation in label.attributes.annotations:
        trichome_type = config.ANNOTATIONS_CLASS_MAPPINGS.get(annotation.category_id, 0)
        if trichome_type in distribution:
            distribution[trichome_type] += 1

In [21]:
distribution

{'clear': 549, 'cloudy': 863, 'amber': 114}

In [13]:
class_distribution

{1: 862, 0: 549, 2: 114}

In [6]:
import numpy as np

# Example confusion matrix
confusion_matrix = np.array([
    [3, 1, 0, 2],  # True 0: 3 TP, 1 Pred as 1, 2 FN
    [1, 4, 2, 1],  # True 1: 4 TP, 1 Pred as 0, 2 Pred as 2, 1 FN
    [0, 2, 5, 1],  # True 2: 5 TP, 2 Pred as 1, 1 FN
    [1, 1, 1, 0]   # FP: 1 FP for each class
])

num_classes = 3

overall_tp = 0
overall_fp = 0
overall_fn = 0

# Compute precision and recall for each class
for i in range(num_classes):
    tp = confusion_matrix[i, i]
    
    # Correctly compute fp by summing the misclassifications for the predicted class
    fp = np.sum(confusion_matrix[:, i]) - tp
    fn = np.sum(confusion_matrix[i, :]) - tp

    overall_tp += tp
    overall_fp += fp
    overall_fn += fn

# Compute overall precision and recall
if overall_tp + overall_fp > 0:
    overall_precision = overall_tp / (overall_tp + overall_fp)
else:
    overall_precision = 0.0

if overall_tp + overall_fn > 0:
    overall_recall = overall_tp / (overall_tp + overall_fn)
else:
    overall_recall = 0.0

print(f"Overall TP: {overall_tp}")
print(f"Overall FP: {overall_fp}")
print(f"Overall FN: {overall_fn}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")


Overall TP: 12
Overall FP: 9
Overall FN: 10
Overall Precision: 0.5714285714285714
Overall Recall: 0.5454545454545454
